In [1]:
import math
import qiskit_qasm2
from qiskit_qasm2 import load, CustomClassical
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from scipy.linalg import expm
from numpy.linalg import norm, matrix_power
from functools import reduce
from itertools import product
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi
from matplotlib.ticker import MaxNLocator
import time

PAULIS = {'I': np.eye(2, dtype='complex'),
          'X': np.array([[0, 1], [1, 0]], dtype='complex'),
          'Y': np.array([[0, -1j], [1j, 0]]),
          'Z': np.diag(np.array([1, -1], dtype='complex'))}


def reverse_mask(x):
    x = ((x & 0x55555555) << 1) | ((x & 0xAAAAAAAA) >> 1)
    x = ((x & 0x33333333) << 2) | ((x & 0xCCCCCCCC) >> 2)
    x = ((x & 0x0F0F0F0F) << 4) | ((x & 0xF0F0F0F0) >> 4)
    x = ((x & 0x00FF00FF) << 8) | ((x & 0xFF00FF00) >> 8)
    x = ((x & 0x0000FFFF) << 16) | ((x & 0xFFFF0000) >> 16)
    return x >> (32 - NQ)

def popcount(n):
    return bin(n).count("1")

def walsh(j:int, k:int):
    brk = reverse_mask(k)
    oddeven = popcount(j & brk) & 1
    if oddeven:
        return -1
    else:
        return 1

def walsh_weight_H(nq: int, gr: int, wl:int, mval:list):
    """
    nq: number qubits                         
    gr: group identificator                   
    wl: walsh id of the basis term            
    mval: matrix elements c_1, ..., c_2^(nq-1) matrix elements assumed positive, automatically assigning -1 to the diagonal terms

    B-matrix structure:
    -c_1  c_2
         -c_2  c_3
                -c_3  c_4

    """
    signbit = 1<<nq
    wlj =  (wl ) & (~signbit)
    if (wl & signbit) > 0: # sign
#     if (wl & signbit): # sign
        swj = -1
    else:
        swj = 1
    ns = 1<<(nq-gr-1)
    nh = 1<<(nq-1)
    sz = 1<<(gr)
    kb1 = math.floor(sz/2.0 - 0.5)
    kb2 = math.floor(sz/2.0 + 0.5)
    wht = 0
    shift_diag = 1
    if (gr == 0): # diagonal terms, flipping sign
        kb2 = kb1 = 0
        swj *= (-1) 
        shift_diag = 0
    for j in range(ns): # for j=0; j < ns; j++:
        wht += swj *(walsh(wlj, j*sz+kb1) + walsh(wlj,j*sz+kb2+nh))*mval[j*sz+kb1+shift_diag]
    
    return wht/signbit

def get_trotter_map_one_step(dict_sets: dict, t: float, order: int):
    """ Compute the trotter map, i.e.
    the result is list of tuples (key, weight) where key is one of the keys from dict_sets
    and weight is resulted weight which arises due to product formula.
    These pairs are repeated the exact number of times as they should in product formula.
    
    dict_sets - dictionary with pairs key : commuting group.
    t - time in resulted exp(tH).
    order = p = 2k - order of the product formula (see formulas (10,11) in Theory of Trotter Error with Commutator Scaling (https://doi.org/10.1103/PhysRevX.11.011020))
    
    """
    if order < 0:
        raise ValueError("Formula order must be 1 or greater")
    if order % 2 == 1 and order != 1:
        raise ValueError("Formula order p = 2k must even number or 1")
    
    if order == 1:
        pairs_list = [(set_id, t)
                      for set_id in dict_sets.keys()]
        return pairs_list[::-1]
    elif order == 2:
        pairs_list = [(set_id, t * 0.5)
                      for set_id in dict_sets.keys()]
        pairs_list[-1] = (pairs_list[-1][0], 2 * pairs_list[-1][1])
        return pairs_list + pairs_list[-2::-1]
    else:
        u_p = 1 / (4 - 4**(1 / (order - 1)))
        tf_1 = get_trotter_map_one_step(dict_sets, u_p*t, order - 2)
        tf_2 = get_trotter_map_one_step(dict_sets, (1-4*u_p)*t, order - 2)
        
        assert tf_2[0][0] == tf_1[-1][0]
        assert tf_2[-1][0] == tf_1[0][0]
        assert tf_1[-1][0] == tf_1[0][0]
        
        tf_2 = [(tf_2[0][0], tf_2[0][1]+tf_1[-1][1])] + tf_2[1:-1] + [(tf_2[-1][0], tf_2[-1][1]+tf_1[0][1])]
        tf_1 = tf_1[:-1] + [(tf_1[-1][0], tf_1[-1][1]*2)] + tf_1[1:]
        
        return tf_1[:-1] + tf_2 + tf_1[1:]

def get_matr_one_step(dict_sets, COEF, p):
    
    number_2q_gates = 0
    number_1q_gates = 0
    two_qubit_gates = set(['cx', 'cz'])
    one_qubit_gates = set(['h', 'rz'])
    
    exp_trotter = np.eye(len(H))
    product_formula_map = get_trotter_map_one_step(dict_sets, COEF, p)
    for k, t_trotter in product_formula_map:
        
        def W(gr, wl):
            nq = int(NQ)
            gr = int(gr)
            wl = int(wl)
            mval = c_map
            wwH = walsh_weight_H(nq, gr, wl, mval)
            return 2*t_trotter*wwH

        customs = [
            CustomClassical("w", 2, W),
        ]

        filename = dict_sets[k]
        circuit = load(filename, custom_classical=customs).reverse_bits()
        dict_gates = circuit.count_ops()
        number_2q_gates +=  sum([v for k,v in dict_gates.items() if k in two_qubit_gates])
        number_1q_gates +=  sum([v for k,v in dict_gates.items() if k in one_qubit_gates])
        matrix = qi.Operator(circuit).data
        exp_trotter = exp_trotter@matrix
        
    total_gates_one_step = number_2q_gates+number_1q_gates
    return exp_trotter, total_gates_one_step


def speed_multiple(b,c_map):
    """Multiply each row of a matrix by given constant."""
    for j, c in enumerate(c_map):
        b[:,j] *= c
    return b

In [ ]:
nq_list = [2,3,4,5,6,7,8,9,10]
p_list = [1,2,4,6]

t = 1
l = 1
c_max = 1
set_error = 10**(-5)

print('Constant speed profile')
print('Parameters: t = {} || c_max = {} || lenght = {}\n'.format(t, c_max, l))

print('------------------------ error={:.2E} ------------------------'.format(set_error))

error_list_p = []
r_list_p = []
one_step_gate_count_list_p = []

for p in p_list:
    print('\n------------------------ order of Trotter formula: {} ------------------------'.format(p))
    error_list = []
    r_list = []
    one_step_gate_count_list = []
    
    r_left = 1
    r_right = 10**6
    
    for nq in nq_list:
        NQ = nq+1
        c_map = [c_max]*(2**nq) # np.random.rand(2**nq)*c_max
        x = np.linspace(0,l,2**(nq))
        dx = x[1]-x[0]

        B = np.eye(2**nq,k=1) - np.eye(2**nq)
        B = speed_multiple(B, c_map)
        H = np.block([[np.zeros_like(B), B],[B.T, np.zeros_like(B)]])
        exp_H = expm(-1j*H*t/dx)

        dict_sets = dict(zip(range(NQ),['decomposition/hdecomp_{}V/hdecomp_{}_{}_GD.qasm'.format(NQ, NQ, k) for k in range(NQ)]))

        time_start = time.time()
        
#         r_left = 1
#         r_right = 10**6
        
        error_fin = None
        print('\n------------------------ Number of qubits: {}+1 ------------------------'.format(nq))
        print('Looking for exact Trotter power from: {} - {}'.format(r_left, r_right))
        while r_left<=r_right:
            r = (r_left + r_right)//2
            COEF = t/r/dx
            exp_trotter, total_gates_one_step = get_matr_one_step(dict_sets, COEF, p)
            exp_trotter = matrix_power(exp_trotter, r)
            error = norm(exp_H - exp_trotter, ord=2)
            
            if r_left == r_right:
                r_right = r
                error_fin = error
                t_gates = total_gates_one_step
                break
                
            if error > set_error:
                r_left = r+1
            else:
                r_right = r
                error_fin = error
                t_gates = total_gates_one_step

        error_list.append(error_fin)
        r_list.append(r_right)
        one_step_gate_count_list.append(t_gates)
        print('Number of gates for one step: {}'.format(total_gates_one_step))
        print('Number of trotter steps = {}'.format(r_right))
        print('Actual error = {}'.format(error_fin))
        print('time to compute: {:.3f} s'.format(time.time() - time_start))
        
        r_left = r_right
        r_right = r_right*5
        
    error_list_p.append(error_list)
    r_list_p.append(r_list)
    one_step_gate_count_list_p.append(one_step_gate_count_list)
    
data_dict = {'p_list':p_list, 'r_list_p':r_list_p, 'one_step_gate_count_list_p':one_step_gate_count_list_p, 'nq_list':nq_list}
np.save('data_dict_eps_10-5_nq-{}-{}_ord-{}-{}_corrected_dx'.format(nq_list[0],nq_list[-1],p_list[0],p_list[-1]),data_dict)

Constant speed profile
Parameters: t = 1 || c_max = 1 || lenght = 1

------------------------ error=1.00E-05 ------------------------

------------------------ order of Trotter formula: 1 ------------------------

------------------------ Number of qubits: 2+1 ------------------------
Looking for exact Trotter power from: 1 - 1000000
Number of gates for one step: 56
Number of trotter steps = 189263
Actual error = 9.999980260657799e-06
time to compute: 0.142 s

------------------------ Number of qubits: 3+1 ------------------------
Looking for exact Trotter power from: 189263 - 946315
Number of gates for one step: 158
Number of trotter steps = 553801
Actual error = 9.999994720207449e-06
time to compute: 0.320 s

------------------------ Number of qubits: 4+1 ------------------------
Looking for exact Trotter power from: 553801 - 2769005
Number of gates for one step: 358
Number of trotter steps = 1292999
Actual error = 9.999994870441351e-06
time to compute: 0.812 s

---------------------

Number of gates for one step: 30274
Number of trotter steps = 53
Actual error = 9.33544531112069e-06
time to compute: 27.960 s

------------------------ Number of qubits: 6+1 ------------------------
Looking for exact Trotter power from: 53 - 265
Number of gates for one step: 63090
Number of trotter steps = 118
Actual error = 9.757193902638867e-06
time to compute: 98.585 s

------------------------ Number of qubits: 7+1 ------------------------
Looking for exact Trotter power from: 118 - 590
Number of gates for one step: 135770
Number of trotter steps = 266
Actual error = 9.785267148307575e-06
time to compute: 462.365 s

------------------------ Number of qubits: 8+1 ------------------------
Looking for exact Trotter power from: 266 - 1330
Number of gates for one step: 287678
Number of trotter steps = 598
Actual error = 9.949167481016739e-06
time to compute: 5970.922 s

------------------------ Number of qubits: 9+1 ------------------------
Looking for exact Trotter power from: 598 - 2